# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [20]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [21]:
df.head(10)

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \r\n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


## LLMChain

In [ ]:
!pip install langchain_community

In [22]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [32]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)


In [35]:
prompt = ChatPromptTemplate.from_template( "Talk about {product}" )

In [36]:

chain = LLMChain(llm=llm, prompt=prompt)

In [37]:
product = 'Queen Size Sheet Set	'
chain.run(product)

"A Queen Size Sheet Set typically includes a flat sheet, a fitted sheet, and two pillowcases designed to fit a Queen-sized bed. These sets are popular choices for many households due to the versatility and comfort they provide.\n\nQueen Size Sheet Sets come in a variety of materials, including cotton, microfiber, linen, and bamboo. Each material offers its own unique benefits, such as breathability, moisture-wicking properties, and durability.\n\nWhen choosing a Queen Size Sheet Set, it is important to consider the thread count, which indicates the number of threads woven into a square inch of fabric. A higher thread count typically results in a softer and more luxurious feel.\n\nAdditionally, Queen Size Sheet Sets are available in a wide range of colors and patterns to suit any decor style. Some sets even come with extra features like deep pockets on the fitted sheet for mattresses of varying thicknesses, or wrinkle-resistant materials for easy care.\n\nOverall, a Queen Size Sheet Set

## SimpleSequentialChain

In [38]:
from langchain.chains import SimpleSequentialChain

In [41]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
     "Give a short and informative description of the product: {product}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [42]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
   "Create a catchy marketing tagline based on this description: {product}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [43]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [44]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
A queen size sheet set includes a fitted sheet, flat sheet, and two pillowcases designed to fit a standard queen size mattress. Made from soft and durable materials such as cotton or microfiber, these sheet sets provide a comfortable and stylish bedding option for your bedroom. With a variety of colors and patterns available, you can easily customize your bedding to suit your personal style.
"Sleep like royalty with our queen size sheet sets - fit for a queen, made for comfort!"

> Finished chain.


'"Sleep like royalty with our queen size sheet sets - fit for a queen, made for comfort!"'

**Repeat the above twice for different products**

## SequentialChain

In [45]:
from langchain.chains import SequentialChain

In [54]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate the following review to English:\n\n{review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key= "english_version"#Give a name to your output
                    )


In [62]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize this English review:\n\n{english_version}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary" #give a name to this output
                    )


In [63]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "What is the original language of this review:\n\n{review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [64]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "The review was in {language} and summarized as: {summary}. Write a polite follow-up message to the customer."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup"
                     )


In [66]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["english_version", "summary", "language", "followup"],
    verbose=True
)

In [67]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'english_version': "I find the taste mediocre. The foam doesn't hold, it's bizarre. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': "The reviewer finds the taste of the product to be mediocre and the foam doesn't hold. They suggest that the product may be an old batch or counterfeit as they have purchased the same product in stores and found the taste to be much better.",
 'language': 'French',
 'followup': 'Dear valued customer,\n\nThank you for taking the time to share your feedback with us. We are sorry to hear that you found the taste of our product to be mediocre and that the foam did not hold up to your expectations. We take all customer feedback seriously and we will investigate this issue further to ensure the quality of our products.\n\nWe would

**Repeat the above twice for different products or reviews**

## Router Chain

In [68]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [69]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [70]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [71]:
llm = ChatOpenAI(temperature=0)

In [73]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [74]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [75]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [76]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [77]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [78]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body is characterized by a continuous spectrum of wavelengths and intensities, which depends only on the temperature of the body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body is proportional to the frequency of the radiation and the temperature of the body."

In [79]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [80]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This genetic information is crucial for maintaining the integrity and stability of our cells and for passing on genetic traits to future generations. \n\nAdditionally, DNA serves as a blueprint for the regulation of gene expression, allowing cells to respond to their environment and to coordinate their activities with other cells in the body. In this way, DNA plays a central role in the complex processes that govern the growth, development, and functioning of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**